In [1]:
import cudf
import numpy as np
import pandas as pd
import plotly.express as px

# init paths to data and models
DATA_PATH = '/data/'

# amount of semantic classes in model
N_SEM_CLUSTERS = 64

#PREFIX = ''
PREFIX = f'{N_SEM_CLUSTERS}/'

MODEL_NAME = 'ft1_ru'

MODEL_PATH = DATA_PATH + f'models/{MODEL_NAME}/'
SEM_MODEL_PATH = DATA_PATH + f'models/sm_{MODEL_NAME}/' + PREFIX
EVENTS_PATH = DATA_PATH + 'events/events_lem.csv'
TMP_PATH = DATA_PATH + f'tmp/sm_{MODEL_NAME}/'
TMP_PATH_FULL = TMP_PATH + PREFIX
LABELS = 'labels.txt'

# plot pandas data frame with columns 'x', 'y', 'label', and 'title' 
def plot_df(df):
    fig = px.scatter(df, x="x", y="y", color='label', hover_name='title')
    fig.show()

'success'

'success'

In [2]:
!mkdir -d {TMP_PATH}
!mkdir -d {TMP_PATH_FULL}
# read words and their semantic labels
with open(SEM_MODEL_PATH + LABELS) as f:
    labels = list(map(lambda l: [l[0], int(l[1][:-1])], [line.split(',') for line in f]))
w2l = {} # word to label
for word, label in labels:
    w2l[word] = label 


# read events
df = pd.read_csv(EVENTS_PATH)
df['description'] = df['description'].apply(lambda s: ' '.join(filter(lambda w: w in w2l, s.split())))
df = df[df['description'] != '']

# calculate semantic vectors for events
X = []
for index, row in df.iterrows():
    vec = np.array([0] * N_SEM_CLUSTERS)
    words = list(filter(lambda w: w in w2l, row.description.split()))
    for word in words:
        vec[w2l[word]] += 1
    vec = vec / len(words)
    X.append(vec)
    if np.any(np.isnan(vec)):
        print(words)
X = np.array(X)

# normalization of semantic vectors
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# calculate distances between events
from scipy.spatial import distance_matrix
dists = distance_matrix(X, X)

# calculate 2d embeding for events
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0, n_jobs=35, early_exaggeration=10, learning_rate=200)
X_2d = tsne.fit_transform(X) 
x_vals, y_vals = list(zip(*X_2d))
df['x'] = x_vals
df['y'] = y_vals

# saving results of this cell
np.save(TMP_PATH_FULL + 'X_norm.npy', X)
df.to_csv(r'' + TMP_PATH_FULL + 'df.csv', index=False)
np.save(TMP_PATH_FULL + 'dists.npy', dists)

print('completed')

mkdir: invalid option -- 'd'
Try 'mkdir --help' for more information.
mkdir: invalid option -- 'd'
Try 'mkdir --help' for more information.


ValueError: invalid literal for int() with base 10: ''